In [ ]:
import keras

import tensorflow as tf

In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor, wait

# Data dạng folder chứa ảnh hoặc âm thanh , text thường có 2 cách load :

   ### Load thẳng dữ liệu ảnh , âm thanh hoặc text vào biến chẳng hạn X_train , Y_train 
        + ưu nhược điêm : 
            - Phụ thuộc số lượng file , kích thước mỗi điểm data
             
             
   ### Load theo batch :
        + Đến batch nào train thì sẽ load data của batch đấy 
    
    - VD :
        
        

In [ ]:
batch_size = 32
epoch = 2

In [ ]:
def process_data(file):
    return np.random.rand((200,161))


In [ ]:
def load_data_full(desc_file):
    data_audio, texts = [], []
    with open(desc_file) as json_line_file:
        for line_num, json_line in enumerate(json_line_file):
            try:
                spec = json.loads(json_line)
                data_audio = process_data(spec['key'])
                texts.append(spec['text'])
            except:
                pass
    return data_audio, texts 

In [ ]:

def next_batch_1(batch_size,data_x,data_y,start):
    x = data_x[start: start + batch_size]
    y = data_y[start: start + batch_size]
    return x,y
    
        


X_train , Y_train = load_data_full('data_small.json')
num_batch = len(X_train) // batch_size

sess= tf.Session()

for i in range(epoch):
    start = 0
    for idx in range(num_batch):
        x, y = next_batch_1(batch_size,X_train,Y_train,start)
        start += batch_size
        sess.run([],feed_dict={......})


Nhược điểm : 
    nếu data X_train ,  Y_train rất lớn sẽ dẫn đến tràn RAM , load chậm ... thời gian bốc batch lâu
    

Viết hàm đơn lẻ này k tốt vì phải truyền nhiều tham số  và quản lí giá trị các tham số truyền vào 


---->> Viết 1 Class cho việc đọc data đồng thời thay đổi cách đọc data

In [ ]:
class Dataset():
    def __init__(self,desc_file,batch_size):
        self.batch_size = batch_size
        self.desc_file = desc_file
        self.start = 0
        
    def load_data(self):
        audio_paths,  texts = [], [], []
        with open(self.desc_file) as json_line_file:
            for line_num, json_line in enumerate(json_line_file):
                try:
                    spec = json.loads(json_line)
                    audio_paths.append(spec['key'])
                    texts.append(spec['text'])
                except:
                    pass
        self.paths = audio_paths
        sefl.texts = durations
        self.num_batch = len(audio_paths) // self.batch_size

    def next_batch_2(batch_size,data_x,data_y,end_epoch):
        
        x_paths = data_x[self.start: self.start + batch_size]
        
        x = [process_data(p) for p in x_paths]
        y = data_y[self.start: self.start + batch_size]
        self.start += self.batch_size
        if end_epoch:
            self.start = 0
            
        return x,y
        

### Code phần mới ta sẽ chỉ xử lí data theo từng batch : hàm process_data sẽ tạo ra data cần thiết cho mình dựa trên path
   Tuy nhiên vẫn phải quản lí batch data dựa theo chỉ số  start và end của epoch 
   -------- 
    Hàm next batch mới 
    

In [ ]:
    def next_batch_2(self):
        idx = 0
        start = 0
        while(idx < self.num_batch):
            x_paths = self.paths[start: start + self.batch_size]
        
            x = [process_data(p) for p in x_paths]
            y = self.texts[start: start + self.batch_size]
            idx += 1
            start += self.batch_size
            yield x, y 

dataset = Dataset(batch_size=32,desc_file='data_small.json')
num_batch = dataset.num_batch

sess= tf.Session()

for i in range(epoch):
    start = 0
    for idx in range(datnum_batch):
        x, y = dataset.next_batch_2()
        
        sess.run([],feed_dict={......})

## Như vậy với mỗi iter model sẽ đợi get 1 batch ra để đưa vào run : 

##  Đã tối ưu chưa ? 

    

# Việc đọc data còn có thể được tốí ưu hơn nữa vơí thread của python :

In [ ]:
    def prepare_data(paths,texts):
        x = [process_data(p) for p in paths]
        y = texts
        return {'x' : x, 'y' : y}
    
    def next_batch(self):
        pool = ThreadPoolExecutor(1)
        future = pool.submit(self.prepare_data,self.paths[:self.batch_size]
                             ,self.texts[:self.batch_size])
        start = self.batch_size
        for i in range(self.num_batch - 1):
            wait([future])
            minibatch = future.result()
            # While the current minibatch is being consumed, prepare the next
            future = pool.submit(self.prepare_data,self.paths[start:self.batch_size]
                             ,self.texts[start:self.batch_size])
            yield minibatch
            start += minibatch_size
        # Wait on the last minibatch
        wait([future])
        minibatch = future.result()
        yield minibatch

# Multi-Gpu with Keras  backend Tensorflow

About Keras  :  Cung cấp 1 higher API cho tensorflow thông qua Model và K.function 
    model = Sequential()

    model.add(Bidirectional(
        LSTM(units=100, return_sequences=True,
            activation='tanh', dropout=0.2,recurrent_dropout=0.5),
        input_shape=(self.sent_length, self.width),
        merge_mode='concat'
    ))

    model.add(Bidirectional(
        LSTM(units=100, return_sequences=True,
            activation='tanh', dropout=0.2,recurrent_dropout=0.5),
        merge_mode='concat'
    ))
    model.add(TimeDistributed(
            Dense(self.num_of_class, activation='softmax')))
            
    optimizer = Adam(lr=1e-3,decay=1e-8)        
    model.compile(loss=....,optimizer=optimizer,metrics=['accuracy'])

About Tensoflow : 
    Cung cấp định nghĩa vùng tính toán qua câu lệnh:
        with tf.device("....")


In [ ]:
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

In [ ]:
# Giới hạn bộ nhớ sử dụng cua GPU

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4

# Có 2 kiểu chính để sử dụng multi-gpu của Tensorflow + Keras:
      
           

# P1:

     Tensor input của model sẽ được slice làm n phần tương ứng n-gpus --> 
     tính được n-output của model và gộp lại thành 1 output --->
     Tính loss trên output này ... update model

# P2: 

    Với mỗi gpu get ra 1 batch input vào model -> output 
    Tính loss + gradients trên output này 
    Tính average gradients của các output ---> sử dụng gradients này làm update

In [ ]:
import keras.layers as L
from keras.layers.core import Lambda
from keras import backend as K
from keras.layers import merge, Concatenate,Flatten
from keras.models import Model,Sequential
from keras.layers import (BatchNormalization, Convolution1D,Convolution2D, Dense,
                          Input, GRU,LSTM, Bidirectional, TimeDistributed,Lambda,Reshape, Activation,Dropout)


In [ ]:
input_dim = 161
output_dim = 91
acoustic_input = Input(shape=(None, input_dim,1), name='acoustic_input')
conv2d_1 = Convolution2D(32, kernel_size = (11,41), strides=(2, 2), padding='valid',
                        data_format="channels_last",activation='relu')(acoustic_input)
conv2d_1 = BatchNormalization(name='bn_conv1_2d')(conv2d_1)



conv2d_2 = Convolution2D(32, kernel_size = (11,21),strides=(1, 2), padding='valid',
                        data_format="channels_last",activation='relu')(conv2d_1)
conv2d_2= BatchNormalization(name='bn_conv2_2d')(conv2d_2)

network_output = Dense(output_dim,activation="softmax",name="output")(conv2d_2)
model = Model(inputs=acoustic_input,outputs=network_output)

model.summary()

In [ ]:
model.outputs

In [ ]:
#code for multi gpu Keras

def make_parallel(model, gpu_count):
    """ Allows a model to run on multiple GPUs.
    """
    def slice_batch(x, n_gpus, part):
        
        sh = K.shape(x)
        L = sh[0] // n_gpus # sh[0] = batch_size

        if part == n_gpus - 1:
            return x[part*L:]

        return x[part*L:(part+1)*L]

    all_outputs = []

    # Empty list for each output in the model
    for i in range(len(model.outputs)):
        all_outputs.append([])

    # Place a copy of the model on each GPU, 
    # each getting a slice of the batch
    with tf.variable_scope(tf.get_variable_scope()):
        
        for i in range(gpu_count):

            with tf.device('/cpu:0'):
                slices = []  # multi-input case
                for x in model.inputs:
                    input_shape = (None,) + tuple(x.get_shape().as_list())[1:]
                    slice_g = Lambda(
                        slice_batch,  # lambda shape: shape,
                        lambda shape: input_shape,arguments={'n_gpus': gpu_count, 'part': i})(x)
                    slices.append(slice_g)

            with tf.device('/gpu:%d' % i):
                with tf.name_scope('tower_%d' % i) as scope:

                    outputs = model(slices)
                    tf.get_variable_scope().reuse_variables()

                    if not isinstance(outputs, list):
                        outputs = [outputs]

                    # Save all the outputs for 
                    # merging back together later
                    for l in range(len(outputs)):
                        all_outputs[l].append(outputs[l])

    # Merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []

        func = lambda x: L.concatenate(x, axis=0)

        for outputs in all_outputs:
            merged.append(func(outputs))

    return Model(inputs=model.inputs, outputs=merged)


In [ ]:
model = make_parallel(model,2)

In [ ]:
model.layers